In [1]:
# 1: creating initial variables

# Create Sagemaker execution role
import sagemaker
from sagemaker.sklearn.estimator import SKLearn
import boto3
import os

role = sagemaker.get_execution_role()
sess = sagemaker.Session()
bucket = sess.default_bucket()

s3_prefix = "script-mode-workflow"
pickle_s3_prefix = f"{s3_prefix}/pickle"
pickle_s3_uri = f"s3://{bucket}/{s3_prefix}/pickle"
pickle_train_s3_uri = f"{pickle_s3_uri}/train"

train_dir = os.path.join(os.getcwd(), "")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
# 1.1: pickling the training data

import pickle
import numpy as np

def pickleTrainingData():
    def unpickle(file):
        with open(file, 'rb') as fo:
            dict = pickle.load(fo, encoding='bytes')
        return dict

    train_data = np.empty((0, 32*32*3))
    train_labels = []

    for i in range(1, 2):
        fileNameDataBatch = './cifar-10-batches-py/data_batch_' + str(i)
        batch = unpickle(fileNameDataBatch)
        train_data = np.vstack((train_data, batch[b'data']))
        train_labels += batch[b'labels']

    train_labels = np.array(train_labels)
    train_data = train_data.reshape(-1, 32, 32, 3) / 255.0
    
    # !!!!! NOTICE HOW THE DATA IS SAVED !!!!!!
    # Will be returned in form of:
    # train_label, train_data  = getDataBack()
    pickle.dump([train_labels,train_data], open('./train.cnn', 'wb'))
    
pickleTrainingData()

In [3]:
# 1.2: getting the test data

def getTestData():
    def unpickle(file):
        with open(file, 'rb') as fo:
            dict = pickle.load(fo, encoding='bytes')
        return dict
    fileNameTestBatch = './cifar-10-batches-py/test_batch'
    test_data = unpickle(fileNameTestBatch)[b'data']
    test_data = test_data.reshape(-1, 32, 32, 3) / 255.0
    test_labels = np.array(unpickle(fileNameTestBatch)[b'labels'])
    
    num_samples_to_select = 600
    random_indices = np.random.choice(test_data.shape[0], num_samples_to_select, replace=False)
    selected_test_data = test_data[random_indices]
    selected_test_labels = test_labels[random_indices]
    
    return selected_test_data, selected_test_labels

test_data, test_labels = getTestData()

In [4]:
# 2: uploading training data to S3

s3_resource_bucket = boto3.Session().resource("s3").Bucket(bucket)
s3_resource_bucket.Object(os.path.join(pickle_s3_prefix, "train.cnn")).upload_file(
    train_dir + "/train.cnn"
)

In [5]:
# 3: creating hyperparameters

# This is not required as these values are the defaults:
hyperparameters = {
    "copy_X": True,
    "fit_intercept": True,
    "normalize": False,
}

In [6]:
# 4: configuring estimator parameters

# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! Modify this based on your script name !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
entry_point = "cnnScript.py"

# Modify this based on your instance type / size
train_instance_type = "ml.m5.large"

estimator_parameters = {
    "entry_point": entry_point,
    "source_dir": "script",
    "framework_version": "0.23-1",
    "py_version": "py3",
    "instance_type": train_instance_type,
    "instance_count": 1,
    "hyperparameters": hyperparameters,
    "role": role,
    "base_job_name": "imagerecognition-model",
}

estimator = SKLearn(**estimator_parameters)

In [7]:
# 5: running the training job

inputs = {
    "train": pickle_train_s3_uri
}

# starting the training job
estimator.fit(inputs)

INFO:sagemaker:Creating training-job with name: imagerecognition-model-2024-11-22-07-38-00-937


2024-11-22 07:38:01 Starting - Starting the training job...
2024-11-22 07:38:29 Starting - Preparing the instances for training...
2024-11-22 07:38:50 Downloading - Downloading input data...
2024-11-22 07:39:20 Downloading - Downloading the training image...
2024-11-22 07:39:56 Training - Training image download completed. Training in progress..2024-11-22 07:39:59,882 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2024-11-22 07:39:59,885 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-11-22 07:39:59,927 sagemaker_sklearn_container.training INFO     Invoking user training script.
2024-11-22 07:40:00,083 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-11-22 07:40:00,095 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-11-22 07:40:00,109 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-11-22 07:

UnexpectedStatusException: Error for Training job imagerecognition-model-2024-11-22-07-38-00-937: Failed. Reason: AlgorithmError: framework error: 
Traceback (most recent call last):
  File "/miniconda3/lib/python3.7/site-packages/sagemaker_containers/_trainer.py", line 84, in train
    entrypoint()
  File "/miniconda3/lib/python3.7/site-packages/sagemaker_sklearn_container/training.py", line 39, in main
    train(environment.Environment())
  File "/miniconda3/lib/python3.7/site-packages/sagemaker_sklearn_container/training.py", line 35, in train
    runner_type=runner.ProcessRunnerType)
  File "/miniconda3/lib/python3.7/site-packages/sagemaker_training/entry_point.py", line 100, in run
    wait, capture_error
  File "/miniconda3/lib/python3.7/site-packages/sagemaker_training/process.py", line 291, in run
    cwd=environment.code_dir,
  File "/miniconda3/lib/python3.7/site-packages/sagemaker_training/process.py", line 208, in check_error
    info=extra_info,
sagemaker_training.errors.E

In [ ]:
# 5.1: getting accuracy of prediction

from sklearn.metrics import accuracy_score
def getAccuracyOfPrediction(cnn_predictions, test_labels):
    cnn_predicted_labels = np.argmax(cnn_predictions, axis=1)
    accuracy = accuracy_score(test_labels, cnn_predicted_labels)
    print("Accuracy:", accuracy)

In [ ]:
# 6: deploying a model to an endpoint

sklearn_predictor = estimator.deploy(initial_instance_count=1,
                                     instance_type='ml.m5.large',
                                     endpoint_name='imagerecognition-endpoint')

getAccuracyOfPrediction(sklearn_predictor.predict(test_data), test_labels)

In [ ]:
# 7: deleting the endpoint

sklearn_predictor.delete_endpoint(True)